In [1]:
from connection import create_spotify_oauth, get_token, sp_get_tracks, sp_get_audio_features, sp_get_audio_analysis
import json
import pandas as pd

In [2]:
sp_ouath = create_spotify_oauth(redirect_uri='localhost:5000/authorize/')
token_info = get_token()

sources = ['5Gyrx7J2IbbQXbbKwa8bca', '04j5JZf6upQSeYnWmcNKrH', '5C9mrfuuQjE6KbceQfGhkj']
all_tracks = []

for source in sources:
   tracks = sp_get_tracks(source, token_info)
   all_tracks.extend(tracks['items'])


In [3]:
def json_response_to_df(tracks):
    data = []
    
    for track in tracks:
        d = {
            'track_id': track['track']['id'],
            'track_name': track['track']['name'],
            'album_id': track['track']['album']['id'],
            'album_name': track['track']['album']['name'],
            'album_date': track['track']['album']['release_date'],
            'artist_ids': [artist['id'] for artist in track['track']['album']['artists']],
            'artist_names': [artist['name'] for artist in track['track']['album']['artists']],
            'artist_number': len(track['track']['album']['artists'][0:]),
            'duration_ms': track['track']['duration_ms'],
            'explicit': track['track']['explicit'],
            'popularity': track['track']['popularity'],
            'preview_url': track['track']['preview_url'],
            'album_cover_640': track['track']['album']['images'][0]['url'],
            'album_cover_300': track['track']['album']['images'][1]['url'],
            'album_cover_64': track['track']['album']['images'][2]['url'],
        }
        
        data.append(d)
    
    return pd.DataFrame(data)

all_tracks_df = json_response_to_df(all_tracks)


In [7]:
def get_audio_features(track_ids):

    track_ids = track_ids.to_string(index=False).split('\n')

    all_tracks_features = []
    
    for track_id in track_ids:
        track_af = sp_get_audio_features(track_id, token_info)
        all_tracks_features.extend(track_af)
        
    return pd.DataFrame(all_tracks_features)

all_tracks_features_df = get_audio_features(all_tracks_df['track_id'])

In [5]:
# with open('all_tracks.json', 'w') as json_file:
#     json.dump(all_tracks, json_file, indent=4)